<a href="https://colab.research.google.com/github/kattens/PubChem-Data-Handler/blob/main/Pubchem_Downloader_Phase_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install PubChemPy

In [71]:
import csv
import pandas as pd
import pubchempy as pcp
import os
import requests

In [72]:
file_path = '/content/drive/MyDrive/cdot_actives_50 1.xlsx'

df = pd.read_excel(file_path)

In [73]:
#based on a experiment theres some NaN values in the column that we should remove
#remove the rows with NaN as the pubchem_cid value
df = df.dropna(subset=['pubchem_cid'])

In [74]:
#access to the pubchem_cid column
df['pubchem_cid']

,pubchem_cid
0,5330175.0
1,5311340.0
2,11511120.0
3,221354.0
4,6806409.0
...,...
60,9829836.0
61,51031035.0
62,452192.0
63,30323.0


In [75]:
#make a list
pubchem_ids = df['pubchem_cid'].tolist() #type = float
pubchem_targets = df['target'].tolist() #type = string

#convert the float to int
pubchem_ids = [int(i) for i in pubchem_ids]

#make every element in pubchem_targets a list and if see | break and make a new entry in the list
for i in range(len(pubchem_targets)):
  if isinstance(pubchem_targets[i], str): #checking if the element is a string before continuing
    if '|' in pubchem_targets[i]:
        pubchem_targets[i] = pubchem_targets[i].strip().split('|')
    else:
        #remove the '' from the entry
        pubchem_targets[i] = pubchem_targets[i].strip().replace(' ', '') #Added strip before replace
        pubchem_targets[i] = [pubchem_targets[i]]

'''
#just to make sure the values are correct to make a dict
print(len(pubchem_ids))
print(len(pubchem_targets))
print(type(pubchem_targets[2][0]))
'''

#make a dictionary such that ids are the keys and targets are the values
pubchem_dict = dict(zip(pubchem_ids , pubchem_targets))
print(pubchem_dict)


{5330175: ['SRC'], 5311340: ['OPRL1'], 11511120: ['EGFR', 'ERBB2', 'ERBB4'], 221354: ['CCR1'], 6806409: ['TP53', 'USP14'], 5329480: ['EGFR', 'ERBB2'], 12947: ['TLR7', 'TLR9'], 444810: ['MRGPRX1'], 135421339: ['BRAF'], 9939609: ['PLA2G7'], 42627755: ['ERNÂ\xa01.00'], 53464483: ['TRPV4'], 3647519: ['HNMT'], 9810709: ['TOP2A'], 6413301: ['TP53', 'USP14'], 119081415: ['CDK7'], 5311382: ['EGFR', 'FGFR1', 'PDGFRB', 'PKMYT1', 'SRC', 'WEE1'], 53315868: ['EHMT2'], 10219: ['RPS2'], 9914412: ['AURKA', 'AURKB'], 2993: ['KCNN1', 'KCNN3'], 24756910: ['EGFR', 'ERBB2'], 6918097: ['ADORA3'], 4534086: ['SLC8A1', 'TRPC3', 'TRPC5', 'TRPC6'], 73416445: ['ATP1A1'], 132928: ['BDKRB2'], 5281035: ['AR'], 121750: ['TYMS'], 9852185: ['BCL2'], 51000408: ['ATR'], 73602827: ['CDK7'], 3499: ['PRKCA', 'PRKCB', 'PRKCD', 'PRKCG', 'PRKCZ'], 9809926: ['CACNA2D1'], 41867: ['CHD1', 'TOP2A'], 6918837: ['HDAC1', 'HDAC2', 'HDAC3', 'HDAC4', 'HDAC6', 'HDAC7', 'HDAC8', 'HDAC9'], 24858111: ['DNMT1', 'DNMT3A', 'DNMT3B'], 33630: ['

# Now we have a dictionary of the ids and targets.


In [ ]:
for cid in pubchem_ids:
  print(cid)

In [77]:
'''
Next step is to download the ids biological test results csv files from pubchem website
We tried to use the PUG API but it wasnt downloading the correct files so we did it manually
'''

#folder to save the csv files in:
folder_path = '/content/drive/MyDrive/IDS_Target_Result'

def fetch_bioassay_results(pubchem_ids, folder_path):
    # Ensure the folder exists
    os.makedirs(folder_path, exist_ok=True)
    for cid in pubchem_ids:
        # The url, using f-string formatting -> same for all the files
        url = f"https://pubchem.ncbi.nlm.nih.gov/sdq/sdqagent.cgi?infmt=json&outfmt=csv&query={{%22download%22:%22*%22,%22collection%22:%22bioactivity%22,%22order%22:[%22acvalue,asc%22],%22start%22:1,%22limit%22:10000000,%22downloadfilename%22:%22pubchem_cid_{cid}_bioactivity%22,%22nullatbottom%22:1,%22where%22:{{%22ands%22:[{{%22cid%22:%22{cid}%22}}]}}}}"

        # Send a GET request to the API
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Define the file path
            file_path = os.path.join(folder_path, f"{cid}.csv")
            # Write the content to a CSV file
            with open(file_path, 'wb') as file:
                file.write(response.content)
            print(f"Bioassay data for CID {cid} has been downloaded and saved to {file_path}")
        else:
            print(f"Failed to retrieve bioassay data for CID {cid}. HTTP Status Code: {response.status_code}")




In [78]:
fetch_bioassay_results(pubchem_ids , folder_path)

Bioassay data for CID 5330175 has been downloaded and saved to /content/drive/MyDrive/IDS_Target_Result/5330175.csv
Bioassay data for CID 5311340 has been downloaded and saved to /content/drive/MyDrive/IDS_Target_Result/5311340.csv
Bioassay data for CID 11511120 has been downloaded and saved to /content/drive/MyDrive/IDS_Target_Result/11511120.csv
Bioassay data for CID 221354 has been downloaded and saved to /content/drive/MyDrive/IDS_Target_Result/221354.csv
Bioassay data for CID 6806409 has been downloaded and saved to /content/drive/MyDrive/IDS_Target_Result/6806409.csv
Bioassay data for CID 5329480 has been downloaded and saved to /content/drive/MyDrive/IDS_Target_Result/5329480.csv
Bioassay data for CID 12947 has been downloaded and saved to /content/drive/MyDrive/IDS_Target_Result/12947.csv
Bioassay data for CID 444810 has been downloaded and saved to /content/drive/MyDrive/IDS_Target_Result/444810.csv
Bioassay data for CID 135421339 has been downloaded and saved to /content/driv

In [ ]:
#THE PRACTICE CODE FOR SEEING IF THE REQUEST RETURNS THE PAGE WE WANT!
'''
import requests
import webbrowser
from IPython.display import IFrame

def fetch_bioassay_results(cid):
    query = f'https://pubchem.ncbi.nlm.nih.gov/compound/{cid}#section=BioAssay-Results&fullscreen=true'

    try:
        # Try opening in a new tab (might not work in all environments)
        webbrowser.open_new_tab(query)
    except Exception as e:
        print(f"Error opening in new tab: {e}")
        # Fallback: Open in default browser (new window or tab)
        webbrowser.open(query)

    # Embed the webpage in the notebook using IFrame
    display(IFrame(query, width=800, height=600))

    # Make the request with 'requests' (optional)
    response = requests.get(query)
    if response.status_code == 200:
        print(f"Successfully accessed the website for CID: {cid}")
    else:
        print(f"Failed to access the website for CID: {cid}, Status code: {response.status_code}")

# Example usage
fetch_bioassay_results(5330175)
'''